In [ ]:
#PyTorch Workflow
what_we_are_covering = {1: "data (prepare and load)",
                        2: "build model",
                        3: "fitting the model to data(training)",
                        4: "making predictions and evaluating a model (inference)",
                        5: "saving and loading a model",
                        6: "putting it all together"}

what_we_are_covering

In [ ]:
import torch
from torch import nn # contains all of pytorch's building block for neural network
import matplotlib.pyplot as plt

torch.__version__

## Data (preparing and loading)

Data can be almost anything in machine learning.

* Excel spreadhseet
* Images of any kind
* Audio
* DNA
* Text

Machine learning is a game of two parts:
1. Get data into a numerical representation
2. Build a mode to learn pattenrs in that numerical representation

To represent this lets create some *known* data using the linear regression formula.

We will use a linear regression formula to make a straight line with *known* parameters

In [ ]:
#create known parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

In [ ]:
len(X), len(y)

### Splitting data into training and test sets


In [ ]:
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

#To better visualie your data

In [ ]:
def plot_predictions(train_data = X_train,
                     train_labels = y_train,
                     test_data = X_test,
                     test_labels = y_test,
                     predictions = None):
  plt.figure(figsize=(10, 7 ))
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  plt.scatter(test_data, test_labels, c="g", s=4, label="Test data")

  if predictions is not None:
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

    plt.legend(prop={"size" : 14});

In [ ]:
plot_predictions()

#building first model


In [ ]:
import torch
from torch import nn
#Create linear regression model class
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1, requires_grad=True))
    self.bias = nn.Parameter(torch.randn(1, requires_grad=True))

    #Forward method to define the compuation in the model
  def forward(self, x:torch.tensor) -> torch.tensor:
    return self.weights * x + self.bias #this is the linear regression formula

### PyTorch model building essentials

* torch.nn - contains all of the buildings for computational graphs (a neural network can be considered a computational graph)
* torch.nn.Parameter - what parameters should our model try and learn, often a pytorch layer from torch.nn will set this for use
* torch.nn.Module - The base class for all neural network modules, if you subclass it, you should overwrite forward()
* torch.optim - this is where the optimiers in Pytorch live, they will help with gradient DeserializationStorageContext
* def forward () - All nn.Module Subclasses require you to overwrite the forward() method

In [ ]:
## Checking the contents of our pytorch model using '.parameters()'

In [ ]:
#Create a random seed
torch.manual_seed(42)

#create an instance of the model
model_0 = LinearRegressionModel()

#check out the parameters
list(model_0.parameters())

In [ ]:
# List named Parameters
model_0.state_dict()

In [ ]:
X_test, y_test

In [ ]:
y_pred = model_0(X_test)
y_pred

# Making predictions with inference mode

In [ ]:
# Make predictions with model
#Inference mode turns off the gradient cause we dont need to keep track
#predictions will be a lot faster cause things are being kept track of

with torch.inference_mode():
  y_preds = model_0(X_test)

y_preds

# You can do something similar with torch.no_grad
with torch.no_grad():
  y_preds = model_0(X_test)

y_preds

In [ ]:
plot_predictions(predictions=y_preds)

## Train model

A way to measure how poorly your predicitons are is to use a loss function

The smaller the better (Mean absolute Error)
Things we need to train
* loss function : A function to measure how wrong your models predictions are
* optimizer : takes into account the loss of a model and adjusts the model's parameters.

And for pytorch we need:
* A training loop
* A testing loop

In [ ]:
# Setup a loss function
loss_fn = nn.L1Loss()

#setup an optimier
optimizer = torch.optim.SGD(params = model_0.parameters(),
                           lr = 0.01) #learning rate, the smaller the learning rate the smaller the change in parameter

### Builing a training loop and a testing loop in Pytorch

A couple of things we need in a training loop:
0. Loop through the data
1. Forward pass (this involves data moving thorugh our model's forward() functions)to make predicitons on data
2. Calculate the loss (compare forward pass predicitions to ground truth labels)
3. Optimizer zero grad
4. Loss backward - move backwards through the network to calculate the gradients of each of the parameters of our model with respect to the loss (** back propagation**)
5. Optimier step - use our project parameters to try and improve the loss (**gradient descent**)



In [ ]:
#An epoch is one loop through the data
epochs = 100

#Track different values
epoch_count = []
lose_values = []
test_loss_values = []
#0, Loop through the data
for epoch in range(epochs):
  # Set the model to training mode
  model_0.train() # train mode in pyTorch sets all parameters that require gradients to require gradient


  #1. Forward pass
  y_pred = model_0(X_train)

  #2. Calculate the loss
  loss = loss_fn(y_pred, y_train)


  #3. Optimizer zero grad
  optimizer.zero_grad()

  #4. Perform backpropagation on the loss with respect to the parameters of the model
  loss.backward()

  #5. Step the optimizer (perform gradient descent)
  optimizer.step()

  ### Testing
  model_0.eval() # turns off different seetings in the model not needed for evaluation/testing (dropout laters)
  with torch.inference_mode():
    #1. Do the forward pass
    test_pred = model_0(X_test)

    #2. Calculate the loss
    test_loss = loss_fn(test_pred, y_test)

  #print whats happening
  if epoch % 10 == 0 :
    print(f"Epoch: {epoch} | Loss: {loss} | Test loss : {test_loss}")
    print(model_0.state_dict())




In [ ]:
model_0.state_dict()

In [ ]:
model_0.state_dict()

##Saving a model in PyTorch
There are three main methods you should know about for saving and loading models in PyTorch.

1. torch.save() - allows you save a PyTorch object in Python's pickle format
2. torch.load() - allows you load a saved PyTorch object
3. torch.nn.Module.load_state_dict() - this allows to load a models saved from dictionary


In [ ]:
#Saving our PyTorch model
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "1_pytorch_workflow_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

MODEL_SAVE_PATH

# 3. Save the model state dict
print(f'Saving model to : {MODEL_SAVE_PATH}')
torch.save(obj=model_0.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
##Loading a PyTorch model
loaded_model_0 = LinearRegressionModel()

In [ ]:
loaded_model_0.state_dict()

In [ ]:
#Make some predicitions with our loaded model
loaded_model_0.eval()
with torch.inference_mode():
  loaded_model_preds = loaded_model_0(X_test)

loaded_model_preds

In [ ]:
#Import Pytorch and matplotlip
import torch
from torch import nn
import matplotlib.pyplot as plt

torch.__version__

In [ ]:
#Setup device agnostic code, use gpu if available else cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## 6.1 Data

In [ ]:
# Create data using linear regression formula of y = weight * X + bias
weight = 0.7
bias = 0.3

# Create range values
start = 0
end = 1
step = 0.02

# Create X and y  (features and labels)
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
X[:10], y[:10]

In [ ]:
#split Data
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
#building a pytorch model
class LinearRegressionModelV2(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_layer  = nn.Linear(in_features=1,
                                   out_features=1) # Changed out_features to 1

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.linear_layer(x) # Apply the linear layer to the input x

In [ ]:
model_1 = LinearRegressionModelV2()
model_1.state_dict()

In [ ]:
# Check the model current device
next(model_1.parameters()).device

In [ ]:
# Set the model to use the target device
model_1.to(device)
next(model_1.parameters()).device

### 6.3 Training
For training we need :
* Loss function
* Optimizer
* Training Loop
* Testing loop

In [ ]:
# Setup loss function
loss_fn = nn.L1Loss() # same as MAE

# Setup our optimizer
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.01)

In [ ]:
# Let's write a training loop
torch.manual_seed(42)

epochs = 200

#Put data on the target device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)


for epoch in range(epochs):
  model_1.train()

  # 1. Forward pass
  y_pred = model_1(X_train)

  # 2. Calculate the loss
  loss = loss_fn(y_pred, y_train)

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. Perform backpropagation
  loss.backward()

  # 5. Optimizer step
  optimizer.step()

  ### Testing
  model_1.eval()
  with torch.inference_mode():
    test_pred = model_1(X_test)

    test_loss = loss_fn(test_pred, y_test)

  # Print out whats happening
  if epoch % 10 == 0:
    print(f'Epoch : {epoch} | Loss {loss} | Test loss : {test_loss}')



In [ ]:
#turn model into evaluation mode
model_1.eval()

#Make predictions on the test data
with torch.inference_mode():
  y_pred = model_1(X_test)

y_pred

In [ ]:
plot_predictions(predictions=y_pred.cpu())

In [ ]:
# Saving and loading a trained model
from pathlib import Path

# 1. Create models directory
MODEL_PATH =  Path('models')
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = '01_pytorch_workflow_model_1.pth'
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. save the model state dict
print(f'Saving model to: {MODEL_SAVE_PATH}')
torch.save(obj=model_1.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
model_1.state_dict()

In [ ]:
#Load a PyTorch

#Create a new instance of linear regression model V2
loaded_model_1 = LinearRegressionModelV2()

#Load the saved model_1 state_dict
